In [1]:
# Subjects
subjects = ['_1']

for s in subjects:
    loaded = io.loadmat('aARI2%s.mat' % s, squeeze_me=True)
    eeg = loaded['aARI2']['eeg8ms'] 

    
eeg.shape[0]

NameError: name 'io' is not defined

In [117]:
# Needed tools
%pylab inline
from scipy import io      
from sklearn.svm import SVC, SVR 
from sklearn.model_selection import cross_val_score, StratifiedShuffleSplit
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, f_classif

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


# Time bin to be concatenated???????
lentimebin = 3
# Number of trials to be averaged
n = 1
# Time window for the epoch
timeStart = 1
timeEnd = 125 

# Define number of time bins and prelocate the scores
nTimeBins = len(np.array(range(timeStart,timeEnd,lentimebin)))
all_scores = np.zeros((len(subjects),nTimeBins))

# loop across subjects
sn = 0
for s in subjects:
    loaded = io.loadmat('aARI2%s.mat' % s, squeeze_me=True)
    eeg = loaded['aARI2']['eeg8ms'] 
    ratio = loaded['aARI2']['ratio']
    direction = loaded['aARI2']['direction']
    resp = loaded['aARI2']['resp']
    crtresp = loaded['aARI2']['crtresp']
    acc = loaded['aARI2']['acc']
   

    ##### Get X and y#####

    X = eeg
    y = ratio
    
    #XX = [row for row in X if row.size != 0]
    XX = []
    yy = []
    
    for count,row in enumerate(X):
        if row.size != 0:
            XX.append(row)
            yy.append(y[count])
        else:
            print(count)
            
    XX = np.array(XX)
    yy = np.array(yy)
    #print("Debug first")
    #print(XX.shape)
    #print(yy.shape)
    #print(y)
    #yy = np.array(yy)
    #print(XX.shape, yy.shape)
    #print(XX[0])
    #print(XX[1])
    #print("===================")
    #print(XX[:,:,1])
    #print(X[:][:][1])
    XXX = []
    for tp in range (1,125):
        temp = np.array([])
        #print("dimen")
        #print(XX[:,:,1].shape)
        for row in XX[:,:,tp]:
            temp = np.append(temp, row)
        XXX.append(temp)
        #print(temp.shape)
    XXX = np.array(XXX)
    print(XXX.shape)
    
    ##### Decode #####
       
    # scaler
    scaler= StandardScaler()
    clf = SVC(C=1, kernel='linear')
    # pipeline
    clf_scal = Pipeline([('scaler', scaler), ('clf', clf)])

    # cross-validation
   # cv = StratifiedShuffleSplit( test_size=0.1, train_size=None, random_state=None, n_iterations=None)  

    cv = StratifiedShuffleSplit(n_splits=5, test_size=0.1, random_state=0)
    cv.get_n_splits(X, y)
    # cv.get_n_splits(XXX, yy)
    
    # inizialize
    scores_AllBin = []
    std_scores_AllBin = []
    # choose time bins
    count = 0 
       
    print('decoding')  

#    for tp in range(timeStart,timeEnd,lentimebin): #(1,125,3)
#        tbin = range(tp,tp+lentimebin) 
#        Xt = X[:, :, tbin]
#        X_long = np.reshape(Xt,(X.shape[0], Xt.shape[1]*lentimebin), order='C')

    for tp in range (1,125):
        tp_eeg = X[:][:][tp] # trial, channel, time
        # tp_eeg = XXX[tp]
        scores_t_X_long = cross_val_score(clf_scal, tp_eeg, y, cv=cv, n_jobs=7)
        scores_AllBin = np.append(scores_AllBin,scores_t_X_long.mean())
        #std_scores_AllBin = np.append(std_scores_AllBin,scores_t_X_long.std())
        count = count +1
        
    # store scores
    all_scores[sn,:] = scores_AllBin
    sn = sn + 1
    
    print ('done')

Populating the interactive namespace from numpy and matplotlib
29
72
79
108
206
215
228
252
280
284
305
329
Debug first
(348, 129, 125)
(348,)
[47 57 67 67 57 47 67 57 67 67 67 67 47 47 47 67 47 67 57 57 57 57 47 57
 67 57 57 47 67 47 47 67 47 57 57 47 67 67 57 67 47 57 67 47 57 67 47 67
 57 47 47 47 47 67 67 57 57 47 47 47 47 47 57 67 57 57 57 57 67 67 57 67
 57 57 57 57 47 57 57 67 47 67 67 67 67 67 67 47 67 67 57 67 47 57 47 47
 47 47 47 57 47 67 57 67 57 47 57 47 57 67 47 47 57 57 57 57 47 67 57 57
 57 67 47 67 47 47 57 47 67 47 47 47 47 57 57 67 67 67 67 57 67 67 67 47
 67 67 47 57 67 47 67 47 47 57 47 57 67 57 67 67 57 57 47 67 47 57 67 57
 57 67 47 47 57 47 67 57 47 47 57 67 67 67 67 47 67 47 47 57 57 57 57 47
 57 67 67 57 67 47 47 67 57 47 57 67 67 47 57 57 57 67 47 47 67 57 47 47
 47 47 47 47 57 57 57 67 57 57 67 57 47 67 67 57 47 57 57 47 47 67 67 67
 67 57 47 67 57 57 67 47 47 67 67 47 57 47 57 57 47 67 47 57 67 67 67 67
 57 47 67 57 67 47 67 47 67 57 47 57 67 47 47 57 57 47

ValueError: Found input variables with inconsistent numbers of samples: [129, 360]

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV


iris_data =pd.read_csv('iris.csv', encoding = 'utf-8')

# 붓꽃 데이터를 레이블과 학습용으로 분리하기
label = iris_data.loc[:, 'Name']
data = iris_data.loc[:, ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth']]

warnings.filterwarnings('ignore')
train_data, test_data, train_label, test_label = (
    train_test_split(data, label, test_size = 0.2, train_size = 0.8, shuffle = True))

# 그리드 서치 객체 생성
parameters = [
    {'C':[1,10,100,1000], 'kernel': ['linear']},
    {'C':[1,10,100,1000], 'kernel': ['rbt'], 'gamma':[0.001, 0.0001]},
    {'C':[1,10,100,1000], 'kernel': ['sigmoid'], 'gamma':[0.001, 0.0001]}
]

kfold_cv = KFold(n_splits = 5, shuffle = True)
clf = GridSearchCV(SVC(), parameters, cv = kfold_cv)

clf.fit(train_data, train_label)
print("최적의 매개변수 ==> ", clf.best_estimator_)
pre = clf.predict(test_data)
print("최종 정답률: ", accuracy_score(test_label, pre))

# GridSearch는 모든 Estimator를 테스트해서 괜찮은 걸 알려줌
# 맥이랑 리눅스의 결과가 다르게 나올 수 있다.

In [110]:
from sklearn import datasets

X, y = datasets.load_iris(return_X_y=True)
print(X.shape)
print(y.shape)

(150, 4)
(150,)
